In [1]:
from sympy import *
import sympy as sp
import numpy as np

# computes the m+1 coefficient given the m1-1 and m2-1 coefficients 
def j_plus(j1, j2, j, m, m1, m2, c_m1minus1, c_m2minus1):
    rhs1 = np.sqrt((j1-m1+1)*(j1+m1))*c_m1minus1
    rhs2 = np.sqrt((j2-m2+1)*(j2+m2))*c_m2minus1
    lhs = np.sqrt((j-m)*(j+m+1))

    return sp.nsimplify((rhs1+rhs2)/lhs)

# computes the m1-1 coefficient given the m+1 and m2-1 coefficients
def j_minus_m1(j1, j2, j, m, m1, m2, c_mplus1, c_m2minus1):
    rhs1 = np.sqrt((j1-m1+1)*(j1+m1))
    rhs2 = np.sqrt((j2-m2+1)*(j2+m2))*c_m2minus1
    lhs = np.sqrt((j-m)*(j+m+1))*c_mplus1

    return sp.nsimplify((lhs-rhs2)/rhs1)

# computes the m-1 coefficient given the m1+1 and m2+1 coefficients 
def j_minus(j1, j2, j, m, m1, m2, c_m1plus1, c_m2plus1):
    rhs1 = np.sqrt((j1+m1+1)*(j1-m1))*c_m1plus1
    rhs2 = np.sqrt((j2+m2+1)*(j2-m2))*c_m2plus1
    lhs = np.sqrt((j+m)*(j-m+1))

    return sp.nsimplify((rhs1+rhs2)/lhs)

In [2]:
# this cell contains the computation of all  
# coefficients for a fixed j = 3
init_printing()
j=3
j1=1
j2=2
m1max = 1
m1min = -1
m2max = 2
m2min = -2

c = sp.zeros(5,3)

for m1 in range(1, -2, -1):
    for m2 in range(2, -3, -1):
        coef = f'c_{j},{m1+m2}_{m1},{m2}='
        row = np.abs(m2-2)
        col = m1+1

        # We verify that the m1 and m2 values do not exceed
        # the max restrictions before assigning m1+1, m1+1
        if m1 is m1max:
            c_m1plus1 = 0
        else:
            c_m1plus1 = c[row, col+1]

        if m2 is m2max:
            c_m2plus1 = 0
        else:
            c_m2plus1 = c[row-1, col]

        # We set the m1=1, m2=2 coefficient to +1
        # by the normalization constraint
        if m1 is 1 and m2 is 2: 
            c[row, col] = 1
            continue
        
        # We apply sequential J- operations to fill
        # up the rest of the table
        c[row, col] = j_minus(j1=j1,j2=j2,j=j,m=(m1+m2+1),m1=m1,m2=m2,
                                c_m1plus1=c_m1plus1,c_m2plus1=c_m2plus1)

pprint(c)





⎡√15   √3       ⎤
⎢───   ──     1 ⎥
⎢ 15   3        ⎥
⎢               ⎥
⎢√5   2⋅√30  √6 ⎥
⎢──   ─────  ── ⎥
⎢5      15   3  ⎥
⎢               ⎥
⎢√10   √15   √10⎥
⎢───   ───   ───⎥
⎢ 5     5     5 ⎥
⎢               ⎥
⎢√6   2⋅√30  √5 ⎥
⎢──   ─────  ── ⎥
⎢3      15   5  ⎥
⎢               ⎥
⎢      √3    √15⎥
⎢ 1    ──    ───⎥
⎣      3      15⎦


In [3]:
# this cell contains the computation of all  
# coefficients for a fixed j = 2
init_printing()
j=2
j1=1
j2=2
m1max = 1
m1min = -1
m2max = 2
m2min = -2
mmax = 2

c = sp.zeros(5,3)

for m1 in range(1, -2, -1):
    for m2 in range(2, -3, -1):
        coef = f'c_{j},{m1+m2}_{m1},{m2}='
        row = np.abs(m2-2)
        col = m1+1

        # We set all coefficients exceeding the maximum
        # m restriction to nan, indicating that they
        # aren't computed
        if abs(m1+m2) > mmax:
            c[row, col] = nan
            continue

        # We verify that the m1 and m2 values do not exceed
        # the max restrictions before assigning m1+1, m1+1
        if m1 is m1max:
            c_m1plus1 = 0
        else:
            c_m1plus1 = c[row, col+1]
        if m2 is m2max:
            c_m2plus1 = 0
        else:
            c_m2plus1 = c[row-1, col]

        # We compute the coefficients differently depending on 
        # the value of m1 (column number)

        # column 3, m1 = 1
        if m1 is 1:
            # We set the m1=1, m2=2 coefficient to +1/sqrt(3)
            # by the normalization constraint (computed on paper)
            if m2 is 1: 
                c[row, col] = 1/sp.sqrt(3)
                continue
            
            # We apply sequential J- operations to fill
            # up the rest of the column
            c[row, col] = j_minus(j1=j1,j2=j2,j=j,m=(m1+m2+1),m1=m1,m2=m2,
                                    c_m1plus1=c_m1plus1,c_m2plus1=c_m2plus1)

        # column 2, m1 = 0
        elif m1 is 0:
            # We use the J+ operation to move diagonally up the
            # cgc table and compute the coefficient associated with
            # m1 = 0, m2 = 2
            if m2 is 2:
                c_m2minus1 = c[row+1, col+1]
                c_mplus1 = 0
                c[row, col] = j_minus_m1(j1=j1,j2=j2,j=j,m=(m1+m2-1),m1=m1,m2=m2,
                                            c_mplus1=c_mplus1,c_m2minus1=c_m2minus1)
                continue
            
            # We apply sequential J- operations to fill
            # up the rest of the column
            c[row, col] = j_minus(j1=j1,j2=j2,j=j,m=(m1+m2+1),m1=m1,m2=m2,
                                        c_m1plus1=c_m1plus1,c_m2plus1=c_m2plus1)

            if abs(c[row, col]) < 1e-5: 
                c[row, col] = 0
        
        # column 1, m1 = -1
        elif m1 is -1:
            # We compute the m1 = -1 m2 = 2 coefficient  
            # by leveraging the normalization condition  
            if m2 is 2:
                c[row, col] = -sqrt(1 - c[row+1, col+1]**2 - c[row+2, col+2]**2)
                continue
            
            # We apply sequential J- operations to fill
            # up the rest of the column
            c[row, col] = j_minus(j1=j1,j2=j2,j=j,m=(m1+m2+1),m1=m1,m2=m2,
                                        c_m1plus1=c_m1plus1,c_m2plus1=c_m2plus1)

pprint(c)


⎡-√3   -√6      ⎤
⎢────  ────  nan⎥
⎢ 3     3       ⎥
⎢               ⎥
⎢-√2   -√6   √3 ⎥
⎢────  ────  ── ⎥
⎢ 2     6    3  ⎥
⎢               ⎥
⎢-√2         √2 ⎥
⎢────   0    ── ⎥
⎢ 2          2  ⎥
⎢               ⎥
⎢-√3    √6   √2 ⎥
⎢────   ──   ── ⎥
⎢ 3     6    2  ⎥
⎢               ⎥
⎢       √6   √3 ⎥
⎢nan    ──   ── ⎥
⎣       3    3  ⎦


In [4]:
# this cell contains the computation of all  
# coefficients for a fixed j = 1
init_printing()
j=1
j1=1
j2=2
m1max = 1
m1min = -1
m2max = 2
m2min = -2
mmax = 1

c = sp.zeros(5,3)

m1 = 1
for m1 in range(1, -2, -1):
    for m2 in range(2, -3, -1):
        coef = f'c_{j},{m1+m2}_{m1},{m2}='
        row = np.abs(m2-2)
        col = m1+1

        # We set all coefficients exceeding the maximum
        # m restriction to nan, indicating that they
        # aren't computed
        if abs(m1+m2) > mmax:
            c[row, col] = nan
            continue
        
        # We verify that the m1 and m2 values do not exceed
        # the max restrictions before assigning m1+1, m1+1
        if m1 is m1max:
            c_m1plus1 = 0
        else:
            c_m1plus1 = c[row, col+1]
        if m2 is m2max:
            c_m2plus1 = 0
        else:
            c_m2plus1 = c[row-1, col]
        
        # We compute the coefficients differently depending on 
        # the value of m1 (column number)

        # column 3, m1 = 1
        if m1 is 1:
            # We set the m1=1, m2=2 coefficient to +1/sqrt(10)
            # by the normalization constraint (computed on paper)
            if m2 is 0: 
                c[row, col] = 1/sp.sqrt(10)
                continue
            
            # We apply sequential J- operations to fill
            # up the rest of the column
            c[row, col] = j_minus(j1=j1,j2=j2,j=j,m=(m1+m2+1),m1=m1,m2=m2,
                                    c_m1plus1=c_m1plus1,c_m2plus1=c_m2plus1)
        # column 2, m1 = 0
        elif m1 is 0:
            # We use the J+ operation to move diagonally up the
            # cgc table and compute the coefficient associated with
            # m1 = 0, m2 = 1
            if m2 is 1:
                c_m2minus1 = c[row+1, col+1]
                c_mplus1 = 0
                c[row, col] = j_minus_m1(j1=j1,j2=j2,j=j,m=(m1+m2-1),m1=m1,m2=m2,
                                        c_mplus1=c_mplus1,c_m2minus1=c_m2minus1)
                continue
            
            # We apply sequential J- operations to fill
            # up the rest of the column
            c[row, col] = j_minus(j1=j1,j2=j2,j=j,m=(m1+m2+1),m1=m1,m2=m2,
                                    c_m1plus1=c_m1plus1,c_m2plus1=c_m2plus1)
        
        # column 1, m1 = -1
        elif m1 is -1:
            # We compute the m1 = -1 m2 = 2 coefficient  
            # by leveraging the normalization condition  
            if m2 is 2:
                c[row, col] = sqrt(1 - c[row+1, col+1]**2 - c[row+2, col+2]**2)
                continue
            
            # We apply sequential J- operations to fill
            # up the rest of the column
            c[row, col] = j_minus(j1=j1,j2=j2,j=j,m=(m1+m2+1),m1=m1,m2=m2,
                                        c_m1plus1=c_m1plus1,c_m2plus1=c_m2plus1)

pprint(c)

⎡√15            ⎤
⎢───   nan   nan⎥
⎢ 5             ⎥
⎢               ⎥
⎢√30  -√30      ⎥
⎢───  ─────  nan⎥
⎢ 10    10      ⎥
⎢               ⎥
⎢√10  -√10   √10⎥
⎢───  ─────  ───⎥
⎢ 10    5     10⎥
⎢               ⎥
⎢     -√30   √30⎥
⎢nan  ─────  ───⎥
⎢       10    10⎥
⎢               ⎥
⎢            √15⎥
⎢nan   nan   ───⎥
⎣             5 ⎦
